In [1]:
import tensorflow as tf

In [3]:
def net():
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters = 6, kernel_size=5, activation='sigmoid', padding='same'),
        tf.keras.layers.AvgPool2D(pool_size=2, strides=2),
        tf.keras.layers.Conv2D(filters = 16, kernel_size=5, activation='sigmoid'),
        tf.keras.layers.AvgPool2D(pool_size=2, strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation = 'sigmoid'),
        tf.keras.layers.Dense(84, activation = 'sigmoid'),
        tf.keras.layers.Dense(10)])

In [4]:
X = tf.random.uniform((1, 28, 28, 1))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output_shape: \t', X.shape)

Conv2D output_shape: 	 (1, 28, 28, 6)
AveragePooling2D output_shape: 	 (1, 14, 14, 6)
Conv2D output_shape: 	 (1, 10, 10, 16)
AveragePooling2D output_shape: 	 (1, 5, 5, 16)
Flatten output_shape: 	 (1, 400)
Dense output_shape: 	 (1, 120)
Dense output_shape: 	 (1, 84)
Dense output_shape: 	 (1, 10)


In [33]:
class TrainCallback(tf.keras.callbacks.Callback):
    def __init__(self, net, train_iter, test_iter, num_epochs):
        self.net = net
        self.train_iter = train_iter
        self.test_iter = test_iter
        self.num_epochs = num_epochs

    def on_epoch_end(self, epoch, logs = None):
        test_acc = self.net.evaluate(self.test_iter, verbose = 0)
        metrics = (logs["loss"], logs["accuracy"], test_acc[1])
        print(f'epoch {epoch}, loss {metrics[0]:.3f}, train acc {metrics[1]:.3f}, 'f'test acc {metrics[2]:.3f}')

        if epoch == self.num_epochs - 1:
            print(f'loss {metrics[0]:.3f}, train acc {metrics[1]:.3f}, 'f'test acc {metrics[2]:.3f}')

In [34]:
def load_data_fashion_mnist(batch_size, resize=None):   #@save
    """Download the Fashion-MNIST dataset and then load it into memory."""
    mnist_train, mnist_test = tf.keras.datasets.fashion_mnist.load_data()
    # Divide all numbers by 255 so that all pixel values are between
    # 0 and 1, add a batch dimension at the last. And cast label to int32
    process = lambda X, y: (tf.expand_dims(X, axis=3) / 255,
                            tf.cast(y, dtype='int32'))
    resize_fn = lambda X, y: (
        tf.image.resize_with_pad(X, resize, resize) if resize else X, y)
    return (
        tf.data.Dataset.from_tensor_slices(process(*mnist_train)).batch(
            batch_size).shuffle(len(mnist_train[0])).map(resize_fn),
        tf.data.Dataset.from_tensor_slices(process(*mnist_test)).batch(
            batch_size).map(resize_fn))

In [35]:
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [36]:
lr, num_epochs, batch_size = 0.9, 10, 256

strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
with strategy.scope():
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    net_model = net()
    net_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
callback = TrainCallback(net_model, train_iter, test_iter, num_epochs)
net_model.fit(train_iter, epochs=num_epochs, verbose=0, callbacks=[callback])

epoch 0, loss 2.320, train acc 0.102, test acc 0.100
epoch 1, loss 1.632, train acc 0.366, test acc 0.585
epoch 2, loss 0.886, train acc 0.649, test acc 0.662
epoch 3, loss 0.717, train acc 0.720, test acc 0.718
epoch 4, loss 0.647, train acc 0.747, test acc 0.763
epoch 5, loss 0.592, train acc 0.771, test acc 0.777
epoch 6, loss 0.545, train acc 0.791, test acc 0.789
epoch 7, loss 0.512, train acc 0.803, test acc 0.795
epoch 8, loss 0.488, train acc 0.815, test acc 0.799
epoch 9, loss 0.470, train acc 0.823, test acc 0.803
loss 0.470, train acc 0.823, test acc 0.803


&lt;tensorflow.python.keras.callbacks.History at 0x2fb9b8c8&gt;